# Sentiment Analysis and Prediction

Sentiment analysis is the process of understanding the opinion of an author about a subject.
Examples include analyzing movie ratings, amazon product reviews or the analysis of Twitter tweet sentiment.

For the purposes of this analysis we will:
- Explore our data
- Transform sentiment carrying columns
- Predict sentiment with a supervised machine learning model

In [ ]:
# %%capture
# !pip install wordcloud

In [ ]:
# Imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import nltk
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import (
    TfidfVectorizer,
    CountVectorizer,
)
from wordcloud import WordCloud
from functools import reduce
from nltk import word_tokenize

nltk.download("punkt")
nltk.download("punkt_tab")

# 1. Load your data
Upload data that has textual value and an indication of the sentiment (0 = negative, 1 = positive)

In [ ]:
# Upload your data as CSV and load as a data frame
df = pd.read_csv('reviews.csv',index_col=0)
print(df.shape)
df.head(100)

# 2. Word cloud and feature creation
Visualize words that carry meaning with a word cloud

In [ ]:
positive_df = df[df["score"] == 1]["review"][:100]  # 1 = positive, 0 = negative
positive_df = reduce(lambda a, b: a + b, positive_df)

# Create and generate a word cloud image
cloud_positives = WordCloud(background_color="white").generate(positive_df)

# Display the generated wordcloud image
plt.imshow(cloud_positives, interpolation="bilinear")
plt.title("Top 100 positive words", y=1.02, size=14)  # Choose title, position and size
plt.axis("off")  # Turn off axis labels

# Don't forget to show the final image
plt.show()

In [ ]:
# Tokenize each item in the review column
word_tokens = [word_tokenize(review) for review in df["review"]]

# Create a new feature for the lengh of each review
df["n_words"] = [len(word_tokens[i]) for i in range(len(word_tokens))]

df

# 3. Building a vectorizer
Use the Tfidf Vectorizer to transform the data into numerical values that can be used to make predictions. 

In [ ]:
# Build the vectorizer
vect = TfidfVectorizer(
    stop_words='english',  # Default list of English stop words
    ngram_range=(1, 2),  # Consider Uni- and Bi-grams
    max_features=200,  # Max number of features
    token_pattern=r"\b[^\d\W][^\d\W]+\b",  # Capture only words using this pattern
)  

vect.fit(df.review)

# Create sparse matrix from the vectorizer
X = vect.transform(df.review)

# Create a DataFrame
df_transformed = pd.DataFrame(data=X.toarray(), columns=vect.get_feature_names_out())
df_transformed

# 4. Building a classifier
Use a logistic regression to predict the sentiment of unseen data. 
Visualize the errors your classifier makes with a confusion matrix.

In [ ]:
dropped = df.drop(["review", "n_words"], axis=1)
transformed = pd.concat([dropped, df_transformed], axis=1)
transformed

In [ ]:
# Define X and y
y = transformed["score"]
X = transformed.drop("score", axis=1)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,  # Set size of test_set
    random_state=456,  # Random seed for reproducibility
)

# Train a logistic regression
log_reg = LogisticRegression().fit(X_train, y_train)

# Predict the labels
y_predicted = log_reg.predict(X_test)

# Print accuracy score and confusion matrix on test set
print("Accuracy on the test set: ", accuracy_score(y_test, y_predicted))
print(confusion_matrix(y_test, y_predicted) / len(y_test))

In [ ]:
ConfusionMatrixDisplay.from_estimator(log_reg, X_test, y_test, normalize="all")
plt.title("Confuson Matrix", y=1.02, size=14)
plt.show()

# 5. Interactive Review Prediction
Now you can test the model with your own reviews! This section allows users to input their own text and get sentiment predictions in real-time.

In [ ]:
def predict_review_sentiment(review_text, vectorizer, model):
    """
    Predict sentiment for a single review
    
    Args:
        review_text (str): The review text to analyze
        vectorizer: Trained TfidfVectorizer
        model: Trained classification model
    
    Returns:
        tuple: (prediction, probability)
    """
    # Transform the input text using the trained vectorizer
    review_vector = vectorizer.transform([review_text])
    
    # Get prediction and probability
    prediction = model.predict(review_vector)[0]
    probability = model.predict_proba(review_vector)[0]
    
    return prediction, probability

In [ ]:
def analyze_user_review():
    """Interactive function to get user input and analyze sentiment"""
    print("=" * 50)
    print("SENTIMENT ANALYZER")
    print("=" * 50)
    print("Enter your review below (or type 'quit' to exit):")
    
    while True:
        user_review = input("\nYour review: ").strip()
        
        if user_review.lower() in ['quit', 'exit', 'q']:
            print("Thanks for using the sentiment analyzer!")
            break
            
        if len(user_review) < 5:
            print("Please enter a longer review (at least 5 characters)")
            continue
        
        # Get prediction
        prediction, probabilities = predict_review_sentiment(user_review, vect, log_reg)
        
        # Format results
        sentiment = "POSITIVE 😊" if prediction == 1 else "NEGATIVE 😞"
        confidence = max(probabilities) * 100
        
        print(f"\n📊 ANALYSIS RESULTS:")
        print(f"   Sentiment: {sentiment}")
        print(f"   Confidence: {confidence:.1f}%")
        print(f"   Positive probability: {probabilities[1]:.3f}")
        print(f"   Negative probability: {probabilities[0]:.3f}")
        print("-" * 30)

In [ ]:
print("🔍 Testing with example reviews:\n")

example_reviews = [
    "This movie was absolutely fantastic! Great acting and amazing plot.",
    "Terrible film, waste of time. Poor acting and boring story.",
    "It was okay, nothing special but not bad either.",
    "Best movie I've seen this year! Highly recommend to everyone.",
    "Worst experience ever, completely disappointed."
]

for i, review in enumerate(example_reviews, 1):
    prediction, probabilities = predict_review_sentiment(review, vect, log_reg)
    sentiment = "POSITIVE 😊" if prediction == 1 else "NEGATIVE 😞"
    confidence = max(probabilities) * 100
    
    print(f"{i}. Review: \"{review[:50]}{'...' if len(review) > 50 else ''}\"")
    print(f"   Prediction: {sentiment} ({confidence:.1f}% confidence)")
    print()

In [ ]:
print("Now try it yourself!")
analyze_user_review()